In [ ]:
import mnist
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import binarize
from skimage.filters import threshold_local
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import cv2
from scipy.ndimage import gaussian_filter

In [ ]:
xtrain = mnist.train_images()
ytrain = mnist.train_labels()

xtest = mnist.test_images()
ytest = mnist.test_labels()

xtrain = xtrain.reshape((-1, 28*28))
xtest = xtest.reshape((-1,28*28))

In [ ]:
# Validációs adathalmaz létrehozása
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.5, random_state=42, stratify=ytrain)

# MLP osztály inicializálása
clf = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(64, 64), max_iter=1)

# Pontosságok tárolása
train_accuracy_list = []
val_accuracy_list = []

# Lépésenkénti tanítás és pontosságok nyomon követése
for i in range(100):
    #print(f'{i + 1} / 100')
    clf.partial_fit(xtrain, ytrain, classes=np.unique(ytrain))
    
    # Pontosság mérése a tanító adathalmazon
    ytrain_pred = clf.predict(xtrain)
    train_accuracy = accuracy_score(ytrain, ytrain_pred)
    train_accuracy_list.append(train_accuracy)
    
    # Pontosság mérése a validációs adathalmazon
    yval_pred = clf.predict(xval)
    val_accuracy = accuracy_score(yval, yval_pred)
    val_accuracy_list.append(val_accuracy)

# Pontosságok megjelenítése
plt.plot(train_accuracy_list, label='Tanító adathalmaz')
plt.plot(val_accuracy_list, label='Validációs adathalmaz')
plt.xlabel('Iterációk')
plt.ylabel('Pontosság')
plt.legend()
plt.show()

In [ ]:
def image_to_bin(img):
    data = list(img.getdata())
    for i in range(len(data)):
        data[i] = 255 - data[i]
    data = np.array(data)/256.0
    return data

In [ ]:
image_folder = "drawn_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        binimg = image_to_bin(image)
        p = clf.predict([binimg])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

In [ ]:
image_folder = "handwritten_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        binimg = image_to_bin(image)
        p = clf.predict([binimg])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Zajszűrés

In [ ]:
def noise_reduction(image_array, sigma):
    return gaussian_filter(image_array, sigma)

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        
        # Zajszűrés a scipy.ndimage Gaussian szűrőjével
        denoised_image = noise_reduction(np.array(image), sigma=155.0)

        # Átalakítjuk a képet szürkeárnyalatossá (grayscale)
        denoised_image_gray = np.array(Image.fromarray(denoised_image).convert("L"))

        p = clf.predict([denoised_image_gray.flatten()])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()


## küszöbölés és zajszűrés

In [ ]:
def noise_reduction(image, kernel_size):
    # Alkalmazzuk a Gauss szűrést a zaj csökkentésére
    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    return blurred_image

def local_thresholding(img):
    # Lokális küszöbölés alkalmazása
    block_size = 35  # A blokk mérete, amelyben a küszöbölés alkalmazódik
    local_threshold = threshold_local(img, block_size, offset=30, method='gaussian')
    binary_image = (img > local_threshold).astype(np.uint8) * 255

    # Kép inverzének előállítása
    inverted_image = cv2.bitwise_not(binary_image)

    return inverted_image

image_folder = "handwritten_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Képek szürkeárnyalatosan beolvasása

        # Zajcsökkentés az OpenCV Gauss szűrőjével
        denoised_image = noise_reduction(image, kernel_size=1)

        # Lokális küszöbölés zajszűrést követően
        binimg = local_thresholding(denoised_image)

        p = clf.predict([binimg.flatten()])
        images.append(binimg)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()


# Gauss filter

In [ ]:
def preprocess_image(image):
    # Zajcsökkentés az OpenCV Gauss szűrőjével
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

    # Lokális küszöbölés zajszűrést követően
    _, binary_image = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    return binary_image

def find_contours(binary_image):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_contours(image, contours):
    image_with_contours = image.copy()
    cv2.drawContours(image_with_contours, contours, -1, (255, 0, 255), 1)
    return image_with_contours

image_folder = "handwritten_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Képek szürkeárnyalatosan beolvasása

        # Kép előfeldolgozása
        processed_image = preprocess_image(image)

        # Kontúrok keresése
        contours = find_contours(processed_image)

        # Kontúrokkal jelölt kép
        image_with_contours = draw_contours(cv2.cvtColor(processed_image, cv2.COLOR_GRAY2RGB), contours)

        # átalakítás szürkeárnyalatos képpé
        image_with_contours_gray = cv2.cvtColor(image_with_contours, cv2.COLOR_RGB2GRAY)

        p = clf.predict([image_with_contours_gray.flatten()])
        images.append(image_with_contours)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()


# Median filter

In [ ]:
def noise_reduction(image, kernel_size):
    # Alkalmazzuk a Medián szűrést a zaj csökkentésére
    return cv2.medianBlur(image, kernel_size)

def preprocess_image(image):
    # Zajcsökkentés a Medián szűrőjével
    denoised_image = noise_reduction(image, kernel_size=1)

    # Lokális küszöbölés zajszűrést követően
    _, binary_image = cv2.threshold(denoised_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    return binary_image

def find_contours(binary_image):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_contours(image, contours):
    image_with_contours = image.copy()
    cv2.drawContours(image_with_contours, contours, 0, (255, 0, 255), 1)
    return image_with_contours

image_folder = "handwritten_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Képek szürkeárnyalatosan beolvasása

        # Kép előfeldolgozása
        processed_image = preprocess_image(image)

        # Kontúrok keresése
        contours = find_contours(processed_image)

        # Kontúrokkal jelölt kép
        image_with_contours = draw_contours(cv2.cvtColor(processed_image, cv2.COLOR_GRAY2RGB), contours)

        # átalakítás szürkeárnyalatos képpé
        image_with_contours_gray = cv2.cvtColor(image_with_contours, cv2.COLOR_RGB2GRAY)

        p = clf.predict([image_with_contours_gray.flatten()])
        images.append(image_with_contours)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

#  Bilaterális szűrő

In [ ]:
def noise_reduction(image, diameter, sigma_color, sigma_space):
    # Alkalmazzuk a Bilaterális szűrést a zaj csökkentésére
    return cv2.bilateralFilter(image, diameter, sigma_color, sigma_space)

def preprocess_image(image):
    # Zajcsökkentés a Bilaterális szűrőjével
    denoised_image = noise_reduction(image, diameter=6, sigma_color=70, sigma_space=70)

    # Lokális küszöbölés zajszűrést követően
    _, binary_image = cv2.threshold(denoised_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    return binary_image

def find_contours(binary_image):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_contours(image, contours):
    image_with_contours = image.copy()
    cv2.drawContours(image_with_contours, contours, -1, (255, 0, 255), 1)
    return image_with_contours

image_folder = "handwritten_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Képek szürkeárnyalatosan beolvasása

        # Kép előfeldolgozása
        processed_image = preprocess_image(image)

        # Kontúrok keresése
        contours = find_contours(processed_image)

        # Kontúrokkal jelölt kép
        image_with_contours = draw_contours(cv2.cvtColor(processed_image, cv2.COLOR_GRAY2RGB), contours)

        # átalakítás szürkeárnyalatos képpé
        image_with_contours_gray = cv2.cvtColor(image_with_contours, cv2.COLOR_RGB2GRAY)

        p = clf.predict([image_with_contours_gray.flatten()])
        images.append(image_with_contours)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()
